# Instacart

## Imports

In [1]:
import os
import sys
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib as plt
import time
import multiprocessing as mp
from datetime import datetime
from tqdm import tqdm


In [2]:
pd.set_option("display.max_rows", None, "display.max_columns", None)


In [2]:
data_path = Path.cwd() / "data"
print(data_path)

c:\Users\Super\Desktop\Python-master\Python_Important\instacart\data


## First Look at Datasets

In [4]:
# datasets = []

# for i in os.listdir(data_path):
#     if i[-4:] == ".csv":
#         datasets.append(i)

# print(datasets)

datasets = sorted([i for i in  os.listdir(data_path) if i[-4:] == ".csv"])
print(datasets)

['aisles.csv', 'departments.csv', 'order_products__prior.csv', 'order_products__train.csv', 'orders.csv', 'products.csv', 'sample_submission.csv']


In [5]:
for i in datasets:
    print(i)
    
    exec(f"{i[:-4]} = pd.read_csv(data_path / i)")
    exec(f"display({i[:-4]}.head())")
    exec(f"print(len({i[:-4]}))")


aisles.csv


,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


134
departments.csv


,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


21
order_products__prior.csv


...

3421083
products.csv


,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


49688
sample_submission.csv


,order_id,products
0,17,39276 29259
1,34,39276 29259
2,137,39276 29259
3,182,39276 29259
4,257,39276 29259


75000


### Understanding how the datasets fit together

In [6]:
for i in datasets:
    print("\n")
    print(i[:-4])
    print(list(eval(f"{i[:-4]}.columns")))
    # print(list(eval(f"{i[:-4]}.unique()")))
    for j in eval(f"{i[:-4]}"):
        print(j)
        #print(list(eval(f"{i[:-4]}{[j]}.unique()")))



aisles
['aisle_id', 'aisle']
aisle_id
aisle


departments
['department_id', 'department']
department_id
department


order_products__prior
['order_id', 'product_id', 'add_to_cart_order', 'reordered']
order_id
product_id
add_to_cart_order
reordered


order_products__train
['order_id', 'product_id', 'add_to_cart_order', 'reordered']
order_id
product_id
add_to_cart_order
reordered


orders
['order_id', 'user_id', 'eval_set', 'order_number', 'order_dow', 'order_hour_of_day', 'days_since_prior_order']
order_id
user_id
eval_set
order_number
order_dow
order_hour_of_day
days_since_prior_order


products
['product_id', 'product_name', 'aisle_id', 'department_id']
product_id
product_name
aisle_id
department_id


sample_submission
['order_id', 'products']
order_id
products


## Join and Merge

In [ ]:
order_products = order_products__prior.append(order_products__train)
order_products =order_products.sample(frac = 0.2)
display(order_products.head())
print(len(order_products))

In [ ]:
orders_and_products = order_products.copy()

In [ ]:
orders_and_products = orders_and_products.merge(products, how = "left",on = "product_id")

In [ ]:
display(orders_and_products[orders_and_products["order_id"] == 1])
print(len(orders_and_products))

In [ ]:
orders_and_products = orders_and_products.merge(aisles, how = "left",on = "aisle_id")
orders_and_products = orders_and_products.merge(departments, how = "left",on = "department_id")


In [ ]:
display(orders_and_products[orders_and_products["order_id"] == 1])
print(len(orders_and_products))

In [ ]:
orders_and_products = orders_and_products.merge(orders, how = "left",on = "order_id")

In [ ]:
display(orders_and_products[orders_and_products["order_id"] == 1])
print(len(orders_and_products))

## Augmenting Data

### Functions

In [ ]:
def find_items_in_order(product_id):
    x = order_products[product_id == order_products["product_id"]]
    orders = list(x["order_id"].unique())
    items = 0
    for i in orders:
        items += len(order_products[i == order_products["order_id"]]["product_id"].unique())
    if len(orders) == 0:
        return 0
    return items / len(orders)



In [ ]:
def find_items_in_order_mp(df, data_path, counter):
    start = time.time()
    idxs = list(df.index)
    # df["counter"] = counter
    for i in idxs:
        
        product_id = df.loc[i, "product_id"]
        x = order_products[product_id == order_products["product_id"]]
        orders = list(x["order_id"].unique())
        items = 0
        for j in orders:
            items += len(order_products[j == order_products["order_id"]]["product_id"].unique())
        if len(orders) == 0:
            df.loc[i, "items_in_order_average"] = 0
        else:
            df.loc[i, "items_in_order_average"] = items / len(orders)
        
        
        
        # print(i, counter)
        if counter < 1 and i % 24 == 0:
            now = time.time()
            print(" \n\n\nthe program has elapsed for (mins): " + str(round(((now - start)/60), 2)))
            print("percentage completed: " + str(round(idxs.index(i) * 100 / len(df))))
            df.loc[i, "counter"] = counter
        
        counter += 1e-8
        
        
        
    # queue.put(results)
    df.to_csv(data_path / "augmented" / f"results_{int(counter)}.csv", index = False)

In [ ]:
def find_order_times(products_info, product_id, index):
    x = order_products[product_id == order_products["product_id"]]
    order_days = list(x["order_dow"])
    for i in order_days:
        products_info.loc[index, f"day_of_week:{i}"] += 1


    order_hours = list(x["order_hour_of_day"])
    for i in order_hours:
        products_info.loc[index, f"hour_of_day:{i}"] += 1
    
    return products_info


### Single Loop

In [ ]:
order_products = order_products__prior.append(order_products__train)
order_products = order_products.sample(frac = 0.2)

products_info = products.copy()
display(products_info.head())
print(len(products_info))

display(order_products.head())
print(len(order_products))

In [ ]:
order_products = order_products.merge(orders, how = "left",on = "order_id")

In [ ]:
display(order_products[order_products["order_id"] == 1])

In [ ]:
products_info["times_ordered"] = 0
products_info["add_to_cart_order_average"] = 0

products_info["days_since_prior_order_average"] = 0
products_info["times_reordered_average"] = 0
products_info["items_in_order_average"] = 0
for i in range(7):
    products_info[f"day_of_week:{i}"] = 0
for i in range(24):
    products_info[f"hour_of_day:{i}"] = 0


In [ ]:
display(products_info.head(15))

In [ ]:
#all lines: 1 min 1 sec
# not last line: 1 min 0.4 sec
# not items in order average: 2.15 sec

In [ ]:
%%time
for i in tqdm(range(len(products_info))):
#     if i == 3:
#         break
    product_id = products_info.loc[i, "product_id"]
    products_info.loc[i, "times_ordered"] = len(order_products.loc[order_products["product_id"] == product_id]["order_id"].unique())
    products_info.loc[i, "add_to_cart_order_average"] = np.mean(order_products.loc[order_products["product_id"] == product_id]["add_to_cart_order"])
    products_info.loc[i, "days_since_prior_order_average"] = np.mean(order_products.loc[order_products["product_id"] == product_id]["days_since_prior_order"])
    products_info.loc[i, "times_reordered_average"] = np.mean(order_products[order_products["product_id"] == product_id]["reordered"])
#     products_info["items_in_order_average"][i] = find_items_in_order(product_id)
    products_info = find_order_times(products_info, product_id, i)


In [ ]:
display(products_info.head())

In [ ]:
print(len(products_info))
test = products_info.sample(frac=0.005)
print(len(test))
test["items_in_order_average"] = test["product_id"].apply(find_items_in_order)
# previous line same as: products_info["items_in_order_average"][i] = find_items_in_order(product_id)
# test["example"] = test["product_id"].apply(lambda x:x += 10)

# test["example"] = test["product_id"] + 10


In [ ]:
products_info.to_csv(data_path / "augmented" / "products_info_part1.csv", index = False)

In [7]:
products_info = pd.read_csv(data_path / "augmented" / "products_info_part1.csv")

### Multiprocessing

In [ ]:
# n_jobs = 10

# for i in range(n_jobs):
#     #launch a task
#     pass

In [8]:
display(products_info.head())

,product_id,product_name,aisle_id,department_id,times_ordered,add_to_cart_order_average,days_since_prior_order_average,times_reordered_average,items_in_order_average,day_of_week:0,day_of_week:1,day_of_week:2,day_of_week:3,day_of_week:4,day_of_week:5,day_of_week:6,hour_of_day:0,hour_of_day:1,hour_of_day:2,hour_of_day:3,hour_of_day:4,hour_of_day:5,hour_of_day:6,hour_of_day:7,hour_of_day:8,hour_of_day:9,hour_of_day:10,hour_of_day:11,hour_of_day:12,hour_of_day:13,hour_of_day:14,hour_of_day:15,hour_of_day:16,hour_of_day:17,hour_of_day:18,hour_of_day:19,hour_of_day:20,hour_of_day:21,hour_of_day:22,hour_of_day:23
0,1,Chocolate Sandwich Cookies,61,19,390,6.151282,10.277473,0.635897,0,40,83,65,45,61,67,29,3,2,3,2,1,1,3,7,16,20,52,32,33,42,37,21,27,30,11,11,8,8,10,10
1,2,All-Seasons Salt,104,13,15,9.733333,6.714286,0.133333,0,3,0,0,2,4,1,5,0,0,0,0,0,0,0,0,1,0,1,1,3,2,0,2,2,1,1,0,1,0,0,0
2,3,Robust Golden Unsweetened Oolong Tea,94,7,48,7.770833,9.333333,0.562500,0,7,16,9,3,5,3,5,1,1,0,0,0,0,1,2,3,6,1,4,6,4,5,3,0,3,2,2,0,3,1,0
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1,71,9.563380,15.552239,0.436620,0,18,13,5,5,9,8,13,2,0,0,0,0,0,0,0,7,4,4,6,5,4,10,6,7,6,2,3,3,0,0,2
4,5,Green Chile Anytime Sauce,5,13,1,12.000000,30.000000,1.000000,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
# When this section starts, restart this cell as it was the messed up one from before
n_jobs = 7
# df_split is type list containg 8 pd.DFs
df_split = np.array_split(products_info, n_jobs)
# queue = mp.Queue()
jobs = []
counter = 0
for i in range(n_jobs):
    p = mp.Process(target=find_items_in_order_mp, args=(df_split[i], data_path, counter))
    jobs.append(p)
    counter += 1
for task in jobs:
    task.start()

for task in jobs:
    task.join()




In [4]:

# results = [queue.get(i) for i in queue]
n_jobs = 7
results = []
for i in range(n_jobs):
    results.append(pd.read_csv(data_path / "augmented" / f"results_{i}.csv"))
    # os.remove(data_path / "augmented" / f"results_{i}.csv")
df = pd.concat(results)
df.sort_values("counter", inplace =  True)

df.to_csv(data_path / "augmented" / "products_info.csv", index=False)
display(df.head())

,product_id,product_name,aisle_id,department_id,times_ordered,add_to_cart_order_average,days_since_prior_order_average,times_reordered_average,items_in_order_average,day_of_week:0,...,hour_of_day:15,hour_of_day:16,hour_of_day:17,hour_of_day:18,hour_of_day:19,hour_of_day:20,hour_of_day:21,hour_of_day:22,hour_of_day:23,counter
0,1,Chocolate Sandwich Cookies,61,19,390,6.151282,10.277473,0.635897,2.848101,40,...,21,27,30,11,11,8,8,10,10,0.000000e+00
24,25,Salted Caramel Lean Protein & Fiber Bar,3,19,419,9.448687,10.829949,0.613365,4.774118,97,...,28,30,21,20,20,12,7,6,8,2.400000e-07
48,49,Vegetarian Grain Meat Sausages Italian - 4 CT,14,20,513,9.056530,12.074534,0.623782,4.265873,115,...,37,36,42,22,23,24,12,11,7,4.800000e-07
72,73,Jasmine Tea Unfiltered Ginger Ale,77,7,5,10.600000,18.000000,0.400000,2.571429,0,...,2,0,0,0,0,0,0,0,1,7.200000e-07
96,97,Organic Chamomile Lemon Tea,94,7,36,8.638889,10.694444,0.277778,4.250000,2,...,3,0,3,1,0,1,1,0,1,9.600000e-07


## Kaggle Style

In [ ]:
order_products = order_products__prior.append(order_products__train)
print(len(order_products))
display(order_products.head())

In [ ]:
orders_df = orders.copy()
print(len(orders_df))
display(orders_df.head())

In [ ]:
# If this cell errors, re run all cells below kaggle style becaue if cell is run twice, duplicate colunm names get appended with _x and _y.
order_products = order_products.merge(products,how="left", on="product_id")
order_products = order_products.merge(departments,how="left", on="department_id")
order_products = order_products.merge(aisles,how="left", on="aisle_id")


In [ ]:
display(order_products.head())

In [ ]:
display(order_products.loc[order_products["order_id"] == 112])

## Functions

In [ ]:
def find_items_in_order_mp(df, data_path, counter):
    start = time.time()
    idxs = list(df.index)
    # df["counter"] = counter
    for i in idxs:
        
        product_id = df.loc[i, "product_id"]
        x = order_products[product_id == order_products["product_id"]]
        orders = list(x["order_id"].unique())
        items = 0
        for j in orders:
            items += len(order_products[j == order_products["order_id"]]["product_id"].unique())
        if len(orders) == 0:
            df.loc[i, "items_in_order_average"] = 0
        else:
            df.loc[i, "items_in_order_average"] = items / len(orders)
        
        
        

        if counter < 1 and i % 24 == 0:
            now = time.time()
            print(" \n\n\nthe program has elapsed for (mins): " + str(round(((start - time)/60), 2)))
            print("percentage completed: " + str(round(idxs.index(i) * 100 / len(df))))
            df.loc[i, "counter"] = counter
        
        counter += 1e-8
        
        
        
    # queue.put(results)
    df.to_csv(data_path / "augmented" / f"results_{int(counter)}.csv")

In [ ]:
def get_items(order_id):
    x = order_products.loc[order_products["order_id"] == order_id]
    lst = []
    for i in x.index:
        lst.append(order_products.loc[i, "product_id"])
    lst = list(sorted(set(lst)))
    lst = [str(i) for i in lst]
    return " ".join(lst)

def get_items_mp(df, data_path, counter):
    df["items"] = ""
    start = time.time()
    df.reset_index(drop=True, inplace=True)
    for i in range(len(df)):
        order_id = df.loc[i, "order_id"]
        x = order_products.loc[order_products["order_id"] == order_id]
        lst = []
        for j in x.index:
            lst.append(df.loc[j, "product_id"])
        lst = list(sorted(set(lst)))
        lst = [str(j) for j in lst]
        df.loc[i, "items"] = " ".join(lst)

        if counter < 1 and i % 24 == 0:
            now = time.time()
            print(" \n\n\nthe program has elapsed for (mins): " + str(round(((now - start)/60), 2)))
            print("percentage completed: " + str(round(i * 100 / len(df))))
            df.loc[i, "counter"] = counter
        
        counter += 1e-8
    
    df.to_csv(data_path / "augmented" / f"kaggle_{int(counter)}.csv", index=False)
        




In [ ]:
print(get_items(112))

In [ ]:
# for loop: 118
test = orders_df.sample(frac=0.001)
test["items"] = ""
print(len(orders_df))
print(len(test))

test["items"] = test["order_id"].apply(get_items)
# for i in test.index:
#     test.loc[i, "items"] = get_items(test.loc[i, "order_id"])



In [ ]:
orders_df = orders_df.sample(frac=0.5)
n_jobs = 7
df_split = np.array_split(orders_df, n_jobs)
jobs = []
counter = 0
for i in range(n_jobs):
    task = mp.Process(target = get_items_mp, args = (df_split[i], data_path, counter))
    jobs.append(task)
    counter += 1





for task in jobs:
    task.start()




for task in jobs:
    task.join()
    

In [ ]:
dfs = []
for i in range(n_jobs):

    dfs.append(pd.read_csv(data_path / "augmented" / f"kaggle_{i}.csv"))
    # os.remove(data_path / "augmented" / f"kaggle_{i}.csv")
df = pd.concat(dfs)
df.to_csv(data_path / "augmented" / "kaggle_orders.csv")

In [ ]:
product_book = order_products.drop_duplicates(subset=["product_id"])
product_book.sort_values(by=["product_id"], inplace = True)
display(product_book.head())